In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 

import torch
from torch.utils.data import Dataset,DataLoader
from torch import nn

from torchvision import transforms

from tqdm import tqdm

In [3]:
particle_df_path = '../data/particle_df.csv'
particle_preproc_df_path = '../data/particle_df_preprocessed.csv'

In [4]:
par_pre_df = pd.read_csv(particle_preproc_df_path)
par_df = pd.read_csv(particle_df_path)
par_df.head()

,eventID,jetID,particleType,particleVx,particleVy,particleVz,particlePx,particlePy,particlePz,particleE,particlePolarPx,particlePolarPy,particlePolarPz,particlePolarE,particlePhi,particleTheta
0,0,0,0,0.000000,0.000000,0.000000,-115.595071,5.513218,107.093643,157.675996,115.726471,0.827630,3.093935,2.347607e-01,3.093935,0.824122
1,0,0,0,0.000000,0.000000,0.000000,-83.072377,4.831796,75.798599,112.561324,83.212776,0.816948,3.083494,5.078805e-01,3.083494,0.831991
2,0,0,-211,-0.981025,1.422285,-33.456345,-11.168506,-8.774579,9.043395,16.838385,14.203125,0.600055,-2.475661,1.395264e-01,-2.475661,1.003814
3,0,0,130,0.073932,0.089866,-2.399344,-8.233158,-1.087632,6.647210,10.637351,8.304688,0.732994,-3.010249,-1.192093e-07,-3.010249,0.895801
4,0,0,-211,0.073905,0.089409,-2.399101,-8.048296,0.478376,6.097900,10.109785,8.062500,0.698202,3.082224,1.395264e-01,3.082224,0.923257


In [5]:
class ParticleDataset(Dataset):
    # The ParticleDataset class inherits the Dataset class and implements the __init__, __len__, and __getitem__ methods

    def __init__(self, path, transform=None):
        # Initializing the ParticleDataset object.
        # "path" is the path to the csv file containing the particle data.
        # "transform" is an optional argument that specifies the transformations to be applied to the data.

        # Read the csv file into a Pandas DataFrame.
        self.x = pd.read_csv(path)

        # Put the coordinates eta and phi as the first two features
        self.x = self.x.reindex(
            columns=["particlePolarPy", "particlePhi"]
            + [
                col
                for col in self.x.columns
                if col not in ["particlePolarPy", "particlePhi"]
            ]
        )

        # Store the "transform" argument.
        self.transform = transform

    def __len__(self):
        """
        Returns the number of particles in the dataset.
        """
        # Return the number of rows in the DataFrame (i.e., the number of particles).
        return self.x.shape[0]

    def __getitem__(self, idx):
        """
        Returns the particles with jetID = idx.
        """
        # Get the rows in the DataFrame that have a "jetID" column equal to "idx".
        x = self.x[self.x.jetID==idx].to_numpy()
        
        # If "transform" was specified, apply it to the data.
        if self.transform:
            x = self.transform(x)
        
        # Return the transformed data.
        return x

In [6]:
# Create a Compose object that applies the "ToTensor" transformation.
train_transform = transforms.Compose([
    transforms.ToTensor(),
])


# Create a ParticleDataset object using the csv file located at "particle_df_path" and the "train_transform" transformations.
train_data = ParticleDataset(particle_df_path, train_transform)

# Access the first element in the dataset to get its shape.
train_data[0].shape

torch.Size([1, 23, 16])

## Testing the Dataset on the KNN function

In [20]:
x = train_data[0]
coord = x[:,:,:2]
knn = []
for i in range(x.shape[1]): #itero su numero di particelle
    deltaEta = x[:,i,1] - x[:,:,1]
    deltaPhi = x[:,i,0] - x[:,:,0]
    distance = torch.sqrt(deltaEta**2 + deltaPhi**2)
    _, index = torch.sort(distance, dim=1, descending=True)
    knn_index = index[0][:10]
    knn.append(knn_index) # appendo li indici dei k nn della particella i-esima
    if i == 5:
        break

knn

[tensor([17, 11, 14,  8,  3, 15, 22, 16, 18,  2]),
 tensor([17, 11, 14,  8,  3, 15, 22, 16, 18,  2]),
 tensor([ 5,  6,  0,  1,  9,  4, 13, 21, 19, 20]),
 tensor([ 5,  6,  0, 13,  1,  4,  9, 21, 19, 20]),
 tensor([17, 11, 14,  3,  8, 15, 22, 16, 18,  2]),
 tensor([17, 11, 14,  8,  3, 15, 22, 16, 18,  2])]

In [36]:
def kNN(x, k):
    ''' input: the jet x and the number of nearest neighbour k
        output: tensor with the nn indeces for each particle, so the output tensor shape will be:  '''
    coords = x[:, :, :2]
    knn_list = []
    for i in range(coords.shape[1]):
        delta_phi = coords[:, i, 0] - coords[:, :, 0]
        delta_eta = coords[:, i, 1] - coords[:, :, 1]
        distances = torch.sqrt(delta_phi**2 + delta_eta**2)
        _, indeces = distances.sort(dim=1, descending=True)
        indeces = indeces[0][:k] # keep only the first k nearest neighbourn
        knn_list.append(indeces)

    # print(knn_list,"\n")
    knn = torch.stack(knn_list, dim=1)
    del knn_list
    
    return knn.t()

x = train_data[3]
# print(x.shape)
kNN(x,10)

tensor([[16, 12, 11, 10,  3,  9, 13,  4, 15, 17],
        [16, 12, 11, 10,  3,  9, 13,  4, 15, 17],
        [16, 10, 12, 11, 13,  9,  3, 15, 17,  4],
        [16, 10, 13,  8, 17, 18,  9, 14, 15, 19],
        [16, 10, 13,  9, 15, 17, 19,  8, 14, 18],
        [16, 11, 12, 10, 13,  9,  3,  4, 15, 18],
        [16,  3, 12,  9, 11, 15, 13, 10,  4, 17],
        [16, 10, 13, 18,  8, 11,  9, 12, 17, 15],
        [16,  3, 12, 11,  4,  9, 15, 19,  7, 17],
        [16, 11, 12,  4, 18,  3,  8,  6,  2,  7],
        [16, 12, 11,  3,  4,  7, 18,  2,  5,  0],
        [16, 13, 10,  9, 15, 17, 19,  8, 14,  5],
        [16, 10, 13,  9, 17, 15, 19,  8, 14, 18],
        [16, 12, 11,  4,  3, 18,  7,  2,  6,  5],
        [16, 12,  3, 11,  4,  9, 15,  7, 19, 13],
        [16, 11, 12,  4, 18,  3,  8,  6, 10,  2],
        [18,  8, 10, 14, 11,  6, 13,  1,  0,  2],
        [16, 11, 12,  4,  3, 18,  7,  8,  6,  2],
        [16,  9, 15,  3, 19, 13, 17, 12, 10,  7],
        [16, 11, 12,  4, 18,  3,  8, 10,  6, 14]])

In [ ]:
def custom_collate(batch):
    """
    A custom collate function that can handle different shape tensors.
    The default collate function provided by PyTorch's DataLoader assumes that all tensors in a batch have the same shape. 
    However, in our case, each "datum" is a set of particles that compose a jet and the number of particles composing a jet is not fixed. 
    Therefore, each tensor representing a jet has a different shape.

    To handle this scenario, we need to override the collate function to be able to stack the tensors into a batch. 
    This function first determines the maximum number of particles among all jets in the batch. 
    Then, it pads all tensors with zeros to make sure they have the same shape. 
    Finally, it stacks the tensors along the batch dimension to return the padded data and original lengths.

    """
    
    # Get the max number of particles among all the jets in the batch
    n_part_max = max(x.shape[1] for x in batch)

    # Pad all the tensors with zeros so they have the same shape
    data = []
    lengths = []
    for x in batch:
        n_part = x.shape[1]
        data.append(torch.cat([x, torch.zeros(1, n_part_max - n_part, 16)], dim=1))
        lengths.append(n_part)

    # Stack the tensors along the batch dimension
    data = torch.stack(data)

    # Return the padded data, original lengths, and target labels
    return data, lengths

In [ ]:
batch_size       = 10
train_dataloader = DataLoader(train_data, batch_size=batch_size, collate_fn=custom_collate)

In [ ]:
# loop over the dataloader to get the data in batches
i=0
for batch, original_length in train_dataloader:
    print(batch.shape, original_length)
    i+=1
    if i==2:
        break

torch.Size([10, 1, 62, 16]) [23, 41, 26, 20, 62, 35, 9, 4, 46, 48]
torch.Size([10, 1, 66, 16]) [41, 22, 2, 40, 21, 26, 25, 66, 28, 2]


In the forward pass of your model, you can use the original lengths to process the data correctly, for example, by masking out the padded zeros.

testing nested tensors

In [ ]:
tensors = []

for i, original_shape in enumerate(original_length):
    # Slice the tensor along the third dimension to get the desired shape
    a = batch[i, :, :original_shape, :]
    tensors.append(a)

In [ ]:
for t in tensors:
    print(t.shape)

torch.Size([1, 41, 16])
torch.Size([1, 22, 16])
torch.Size([1, 2, 16])
torch.Size([1, 40, 16])
torch.Size([1, 21, 16])
torch.Size([1, 26, 16])
torch.Size([1, 25, 16])
torch.Size([1, 66, 16])
torch.Size([1, 28, 16])
torch.Size([1, 2, 16])


In [ ]:
nested = torch.nested.nested_tensor(tensors)

AttributeError: module 'torch' has no attribute 'nested'

In [ ]:
nested[2].shape

torch.Size([1, 2, 16])